# Исследование надежности заемщиков.

<b>Описание проекта</b> 

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Описание данных:
<br>children — количество детей в семье
<br>days_employed — общий трудовой стаж в днях
<br>dob_years — возраст клиента в годах
<br>education — уровень образования клиента
<br>education_id — идентификатор уровня образования
<br>family_status — семейное положение
<br>family_status_id — идентификатор семейного положения
<br>gender — пол клиента
<br>income_type — тип занятости
<br>debt — имел ли задолженность по возврату кредитов
<br>total_income — ежемесячный доход
<br>purpose — цель получения кредита

### Шаг 1. Обзор данных

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Шаг 2.1 Заполнение пропусков

Выводы по результатам первичного осмотра таблицы:

<br>В столбце 'days_employed' значения положительные и отрицательные (причина не ясна);
<br>В столбцах 'days_employed' и 'total_income' есть отсутствующие данные, их количество совпадает;
<br>В столбцах 'days_employed' и 'total_income' много цифр после запятой (причина не ясна);
<br>В столбце 'education' данные в разном регистре;
<br>В столбце 'purpose' цель кредита указана абсолютно по разному (даже одна и та же цель может быть записана иначе);

In [2]:
data_lines = data['children'].count()
days_empty_lines = data['children'].count() - data['days_employed'].count()
empty_lines_percent = days_empty_lines / data_lines
print('Количество строк:', data_lines)
print('Количество пустых строк:', days_empty_lines)
print('Процент пустых строк: {:.1%}'.format(empty_lines_percent))

Количество строк: 21525
Количество пустых строк: 2174
Процент пустых строк: 10.1%


<b>Выводы по количеству пустых строк:</b>

Пустые данные занимают 10% всей информации, что много и, в случае удаления (игнорирования) этих данных есть риск большой погрешности при ответе на поставленные вопросы. Возможные причины пропусков:
- техническая ошибка;
- остутствие стабильного дохода или дохода в целом (пенсионеры, безработные);
- остутствие стажа (студенты);
- нежелание указывать данные.

<b>Решение для столбца с общим доходом:</b>
- удалить данные или заменить на 0 (плохое решение, грозит потерей большого объема информации)
- поставить средние значения (плохое решение из-за большой разницы в значениях)
- медианные значения (лучшее решение, так как намного точнее отображают действительность, участвуют в подсчете и уменьшают погрешность в вычислениях)

### Шаг 2.2 Проверка данных на аномалии и исправления.

Для проверки аномалий проверим уникальные данные в каждом столбце. 

Начнем со столбца "children" - дети:

In [3]:
print(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Аномалии: двадцать детей (все, конечно, возможно, но...) и отрицательное количество детей (-1). Похоже, что кто-то ставил тире при указании количества детей, в результате чего получилось отрицательное значение, цифра "20" могла появится в результате технической ошибки или при некорректном заполнении данных. Предлагается заменить 20 детей на 2, а -1 на 1:

In [4]:
data.loc[data['children'] == -1, 'children'] = 1
data.loc[data['children'] == 20, 'children'] = 2
print(data.groupby('children')['children'].count()) 

children
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


Аномалии в столбце "children" исправлены. Далее рассматриваем аномалии в столбце "days_employed". Уже по первым строкам понятно, что некоторых дней стажа подозрительно много, но, чтобы подтвердить информацию, можно вывести максимальное и минимальное значение, предварительно исправив все отрицательные значения на положительные, так как стаж не может быть отрицательным. 

In [5]:
data['days_employed'] = abs(data['days_employed'])
data['days_employed'] = data['days_employed'] / 365
data = data.rename(columns={'days_employed': 'years_employed'})
print('Стаж (минимальный):', data['years_employed'].min())
print('Стаж (максимальный):', data['years_employed'].max())

Стаж (минимальный): 0.06614146093282515
Стаж (максимальный): 1100.6997273296713


Узнаем максимальный возраст клиента:

In [6]:
print('Возраст клиента (максимальный):', data['dob_years'].max())

Возраст клиента (максимальный): 75


Можно сделать вывод, что аномальными являются данные по дням работы свыше 57 лет, берем за истину, что стаж считается с 18 лет. Проверяем количество аномальных данных:

In [7]:
anomaly_years_employed = data[data['years_employed'] > 57]
display(anomaly_years_employed.head(5))
display(anomaly_years_employed['income_type'].value_counts())

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,932.235814,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,1096.660649,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,927.539597,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,996.023258,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,919.401832,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью


пенсионер      3443
безработный       2
Name: income_type, dtype: int64

Как мы видим, все аномальные значения принадлежат пенсионерам и безработным. Причина некорректных данных неизвестна, но очень похоже, что при обработке данных произошла техническая ошибка и данные все еще указаны в днях. Попробуем исправить ситуацию:

In [8]:
def correct_employed_years(data):
    if data['years_employed'] > 57:
        return data['years_employed'] / 24
    else:
        return data['years_employed']
data['years_employed'] = data.apply(correct_employed_years, axis=1)
data.head(5)

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,15.406637,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,38.843159,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Теперь данные похожи на правду: стаж 11 лет при возрасте 36 лет или 23 года при возрасте 42 года. Теперь можно заполнить пустые значения медианными данными:

In [9]:
data['years_employed'] = data['years_employed'].fillna(data['years_employed'].median())

Далее рассматриваем аномалии в столбце "dob_years", в котором указан возраст клиента. Мы уже знаем, что максимальное значение - 75, теперь узнаем минимальное значение:

In [10]:
print('Возраст клиента (минимальный):', data['dob_years'].min())

Возраст клиента (минимальный): 0


Похоже на аномалию, связанную с тем, что кто-то не указал свой возраст. Проверим общее количество строк с клиентами, чей возраст менее 18 лет:

In [11]:
data[data['dob_years'] < 18].count()

children            101
years_employed      101
dob_years           101
education           101
education_id        101
family_status       101
family_status_id    101
gender              101
income_type         101
debt                101
total_income         91
purpose             101
dtype: int64

101 строка - можно либо заменить данные медианным значением, а можно удалить данные, так как сильно на статистику они не повлияют. Удаление:

In [12]:
data = data[data['dob_years'] >= 18].reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21424 entries, 0 to 21423
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21424 non-null  int64  
 1   years_employed    21424 non-null  float64
 2   dob_years         21424 non-null  int64  
 3   education         21424 non-null  object 
 4   education_id      21424 non-null  int64  
 5   family_status     21424 non-null  object 
 6   family_status_id  21424 non-null  int64  
 7   gender            21424 non-null  object 
 8   income_type       21424 non-null  object 
 9   debt              21424 non-null  int64  
 10  total_income      19260 non-null  float64
 11  purpose           21424 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Далее рассматриваем аномалии в столбце "education", в котором указано образование:

In [13]:
print(data['education'].unique())

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']


В целях исключения аномалии, которые, скорее всего, возникли из-за отсутствия указания необходимого формата для заполнения или человеческой невнимательности, приведем значения к нижнему регистру и выведем значения на экран, чтобы убедиться, что аномалии исправлены:

In [14]:
data['education'] = data['education'].str.lower()
print(data['education'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


Проверяем аномалии в столбце "education_id", значения, очевидно, должны соответствовать столбцу "education":

In [15]:
print(data['education_id'].value_counts())

print(data['education'].value_counts())

1    15169
0     5225
2      742
3      282
4        6
Name: education_id, dtype: int64
среднее                15169
высшее                  5225
неоконченное высшее      742
начальное                282
ученая степень             6
Name: education, dtype: int64


Значения совпадают, теперь проверяем значения в столбце "family_status" и сверяем со столбцом "family_status_id":

In [16]:
print(data['family_status'].value_counts())
print(data['family_status_id'].value_counts())

женат / замужем          12331
гражданский брак          4156
Не женат / не замужем     2797
в разводе                 1185
вдовец / вдова             955
Name: family_status, dtype: int64
0    12331
1     4156
4     2797
3     1185
2      955
Name: family_status_id, dtype: int64


Значения совпадают, каждому статусу соответствует своя цифра в id, теперь проверяем значения в столбце "gender":

In [17]:
print(data['gender'].value_counts())

F      14164
M       7259
XNA        1
Name: gender, dtype: int64


Значение XNA не влияет на общее значение, причины указания такого значения неизвестны и не влияют на данные. Следующая проверка касается столбца "income_type":

In [18]:
print(data['income_type'].unique())

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


Аномалий в этом столбце нет. Переходим к столбцу "debt":

In [19]:
print(data['debt'].value_counts())

0    19691
1     1733
Name: debt, dtype: int64


Аномалий в этом столбце нет. Переходим к столбцу "total_income". Заполним его медианными значениями, так как средние не отобразят реальную картину при наличии большого разброса дохода:

In [20]:
data['total_income'] = data['total_income'].fillna(data['total_income'].median())

Переходим к столбцу "purpose":

In [21]:
print(data['purpose'].value_counts())

свадьба                                   792
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     649
операции с жильем                         647
жилье                                     641
покупка жилья                             641
покупка жилья для семьи                   640
строительство собственной недвижимости    633
операции со своей недвижимостью           630
недвижимость                              630
строительство жилой недвижимости          623
покупка недвижимости                      621
строительство недвижимости                620
покупка своего жилья                      619
ремонт жилью                              610
покупка жилой недвижимости                604
на покупку своего автомобиля              502
заняться высшим образованием      

Как мы видим, многие одинаковые данные записаны по разному (сделка с автомобилем = приобретение автомобиля или получение образования = профильное образование), скорее всего данные потребуется объединить по общему признаку.

### Шаг 2.3. Изменение типов данных.

Заменим данные в столбцах 'years_employed' и 'total_income' на целочисленные и проверим результат:

In [22]:
data['years_employed'] = data['years_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21424 entries, 0 to 21423
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21424 non-null  int64 
 1   years_employed    21424 non-null  int64 
 2   dob_years         21424 non-null  int64 
 3   education         21424 non-null  object
 4   education_id      21424 non-null  int64 
 5   family_status     21424 non-null  object
 6   family_status_id  21424 non-null  int64 
 7   gender            21424 non-null  object
 8   income_type       21424 non-null  object
 9   debt              21424 non-null  int64 
 10  total_income      21424 non-null  int64 
 11  purpose           21424 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

Посчитаем количество дубликатов:

In [23]:
data.duplicated().sum()

71

Удаляем дубликаты и проверяем:

In [24]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим сводную таблицу 'education' и 'education_id':

In [25]:
data_education = data.pivot_table(index=['education'], values = 'education_id').sort_values(by='education_id')
display(data_education)

,education_id
education,
высшее,0
среднее,1
неоконченное высшее,2
начальное,3
ученая степень,4


Создадим сводную таблицу 'family_status' и 'family_status_id':

In [26]:
data_family_status = data.pivot_table(index=['family_status'], values = 'family_status_id').sort_values(by='family_status_id')
display(data_family_status)

,family_status_id
family_status,
женат / замужем,0
гражданский брак,1
вдовец / вдова,2
в разводе,3
Не женат / не замужем,4


Удалим из исходного датафрейма столбцы education и family_status, оставив только их идентификаторы: education_id и family_status_id:

In [27]:
data = data.drop(['family_status', 'education' ], axis = 1)
display(data)

,children,years_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,11,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,15,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,11,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,38,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...
21348,1,12,43,1,1,F,компаньон,0,224791,операции с жильем
21349,0,39,67,1,0,F,пенсионер,0,155999,сделка с автомобилем
21350,1,5,38,1,1,M,сотрудник,1,89672,недвижимость
21351,3,8,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Шаг 2.6. Категоризация дохода.

Cоздаем столбец total_income_category с категориями:

In [28]:
def category_income(income):
    total_income = income['total_income']
    if total_income <= 30000:
        return 'E'
    if total_income >= 30001 and total_income <= 50000:
        return 'D'
    if total_income >= 50001 and total_income <= 200000:
        return 'C'
    if total_income >= 200001 and total_income <= 1000000:
        return 'B'
    if total_income >= 1000001:
        return 'A'
    
data['total_income_category'] =  data.apply(category_income, axis=1) 
display(data)

,children,years_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,23,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,11,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,15,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,11,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,38,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
...,...,...,...,...,...,...,...,...,...,...,...
21348,1,12,43,1,1,F,компаньон,0,224791,операции с жильем,B
21349,0,39,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C
21350,1,5,38,1,1,M,сотрудник,1,89672,недвижимость,C
21351,3,8,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B


### Шаг 2.7. Категоризация целей кредита.

Создаем функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category:

In [29]:
def purpose_category(purpose):
    if  'авто' in purpose:
        return 'операции с автомобилем'
    elif 'недвиж' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образо' in purpose:
        return 'получение образования'
    else:
        return 'нет категории'

In [30]:
data['purpose_category'] = data['purpose'].apply(purpose_category)
display(data.head(5))

,children,years_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,23,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,11,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,15,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,11,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,38,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


### Ответы на вопросы.

##### Вопрос 1:
Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [31]:
children_and_debt = data.groupby('children')['debt'].agg(['count', 'sum'])
children_and_debt['debt_percent'] = (children_and_debt['sum'] / children_and_debt['count']) * 100
display(children_and_debt)

,count,sum,debt_percent
children,,,
0,14022,1058,7.545286
1,4839,442,9.134119
2,2114,202,9.555345
3,328,27,8.231707
4,41,4,9.756098
5,9,0,0.000000


Судя по процентам просроченных кредитов, чаще всего не возвращают кредиты люди с 1,2 и 4 детьми, меньше всего - бездетные. Судя по всему, отсутствие детей позволяет людям оставлять больше денег на выплату кредита.

##### Вопрос 2:
Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [32]:
family_and_debt = data.groupby('family_status_id')['debt'].agg(['count', 'sum'])
family_and_debt['debt_percent'] = (family_and_debt['sum'] / family_and_debt['count']) * 100
data_family_status.merge(family_and_debt, on='family_status_id')
display(family_and_debt)
display(data_family_status)

,count,sum,debt_percent
family_status_id,,,
0,12290,927,7.542718
1,4130,386,9.346247
2,954,62,6.498952
3,1185,85,7.172996
4,2794,273,9.770938


,family_status_id
family_status,
женат / замужем,0
гражданский брак,1
вдовец / вдова,2
в разводе,3
Не женат / не замужем,4


Судя по процентам просроченных кредитов, чаще всего не возвращают кредиты люди, которые не состоят в браке или находятся в гражданском браке. Наименьший процент просроченных кредитов у вдов и вдовцов. 

##### Вопрос 3:
Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [33]:
income_and_debt = data.groupby('total_income_category')['debt'].agg(['count', 'sum'])
income_and_debt['percent'] = (income_and_debt['sum'] / income_and_debt['count']) * 100
display(income_and_debt)
print('Расшифровка категорий дохода: 1000001 и выше — A, 200001–1000000 — B, 50001–200000 — C, 30001–50000 — D, 0–30000 — E')

,count,sum,percent
total_income_category,,,
A,25,2,8.000000
B,5015,355,7.078764
C,15943,1353,8.486483
D,348,21,6.034483
E,22,2,9.090909


Расшифровка категорий дохода: 1000001 и выше — A, 200001–1000000 — B, 50001–200000 — C, 30001–50000 — D, 0–30000 — E


Данные показывают, что чаще всего не возвращают кредиты люди со средним и низким уровнем дохода.

##### Вопрос 4:
Как разные цели кредита влияют на его возврат в срок?

In [34]:
purpose_and_debt = data.groupby('purpose_category')['debt'].agg(['count', 'sum'])
purpose_and_debt['percent'] = (purpose_and_debt['sum'] / purpose_and_debt['count']) * 100
display(purpose_and_debt)

,count,sum,percent
purpose_category,,,
операции с автомобилем,4284,400,9.337068
операции с недвижимостью,10764,779,7.237087
получение образования,3995,370,9.261577
проведение свадьбы,2310,184,7.965368


Данные показывают, что чаще всего не возвращают кредит люди, которые берут его для оплаты образования или операциями с автомобилем.

## Общий вывод:

Исходя из вышеизложенных данных, можно составить вывод о самом надежном и самом ненадежном клиенте банка. 

Самый надежный: клиент без детей, вдовец/вдова, с высоким уровнем дохода, берущий кредит на покупку недвижимости.

Самый ненадежный: клиент с 4 детьми, не состоящий в браке, с низким уровнем дохода и берущие кредит на автомобиль.